In [ ]:
import os
import numpy as np
import astropy
import matplotlib
import scipy
import pandas as pd
import wdwarfdate as wd #needs to be cloned from github


ModuleNotFoundError: No module named 'wdwarfdate'

In [2]:
os.chdir('/Users/mlclark/Library/CloudStorage/GoogleDrive-mclark.astro@gmail.com/My Drive/UMa/final_uma/wdwarf_files')
#change to the final_uma wdwarf_files folder in your directory

In [3]:
#these are the 201 wd pm members that were in the vincent 2024 catalog. if doing something else, make sure to replace that file below
wd_cands=pd.read_csv('25_6_30_wds_w_ages.csv', dtype={'GaiaID':str}) #ensures your gaia ids don't get turned to scientific notation

In [4]:
#Getting WD ages
#split by da and non-da (specfified in vincent catalog). since the system is young and there are a lot more massive stars, i use the parsec models
da_srcs=wd_cands[wd_cands.SpType=='DA'].reset_index(drop=True)
nonda_srcs=wd_cands[wd_cands.SpType!='DA'].reset_index(drop=True)

In [ ]:
WDda = wd.WhiteDwarf(da_srcs.Teff.values,da_srcs.e_Teff.values,da_srcs.logg.values,da_srcs.e_logg.values,
                           model_wd='non-DA',feh='p0.00',vvcrit='0.0',
                           model_ifmr = 'Cummings_2018_PARSEC',
                           high_perc = 84, low_perc = 16,
                           datatype='Gyr',
                           save_plots=False, display_plots=False)  #i'm going to edit them so I don't want to save/see them so many times or save them
WDda.calc_wd_age()

In [ ]:
WDnon = wd.WhiteDwarf(nonda_srcs.Teff.values,nonda_srcs.e_Teff.values,nonda_srcs.logg.values,nonda_srcs.e_logg.values,
                           model_wd='non-DA',feh='p0.00',vvcrit='0.0',
                           model_ifmr = 'Cummings_2018_PARSEC',
                           high_perc = 84, low_perc = 16,
                           datatype='Gyr',
                           save_plots=False, display_plots=False)  #i'm going to edit them so I don't want to save/see them so many times or save them
WDnon.calc_wd_age()

In [ ]:
da_results=WDda.results.to_pandas()
nonda_results=WDnon.results.to_pandas() #this is so you can merge the ages into the other file without writing them in manually

In [ ]:
nonda_results['GaiaID']=nonda_srcs['GaiaID'].values  #to be able to merge on gaia ids. as long as you didn't reorder somehow, this should be the right order that's in the results table
da_results['GaiaID']=da_srcs['GaiaID'].values

In [ ]:
wd_nonda_res=pd.merge(nonda_srcs, nonda_results[['GaiaID',
       'cooling_age_median', 'cooling_age_err_low', 'cooling_age_err_high',
       'total_age_median', 'total_age_err_low', 'total_age_err_high',
       'initial_mass_median', 'initial_mass_err_low', 'initial_mass_err_high']], how='left', on='GaiaID')
wd_da_res=pd.merge(da_srcs, da_results[['GaiaID',
       'cooling_age_median', 'cooling_age_err_low', 'cooling_age_err_high',
       'total_age_median', 'total_age_err_low', 'total_age_err_high',
       'initial_mass_median', 'initial_mass_err_low', 'initial_mass_err_high']], how='left', on='GaiaID')

In [ ]:
wd_ages=pd.concat([wd_da_res, wd_nonda_res])  #should be the same length as your original file!
len(wd_ages)

In [ ]:
#save your files!
#wd_ages.to_csv('file_name.csv', index=False)

In [ ]:
joint_da=wd_da_res[(wd_da_res.total_age_median<.750)&((wd_da_res.total_age_median>.300))].reset_index(drop=True)
joint_nonda=wd_nonda_res[(wd_nonda_res.total_age_median<.750)&((wd_nonda_res.total_age_median>.300))].reset_index(drop=True)

In [ ]:
data=[] #this is so i can store the age data to edit my plots to be how I want. recommend only doing this for the ones you actually want to edit - ie the ones in an age range of interest. make sure there are no nans in here!! 
for i in range(len(joint_da)):
    WD=wd.WhiteDwarf(joint_da.Teff.values,joint_da.e_Teff.values,joint_da.logg.values,joint_da.e_logg.values,
                            model_wd='DA',feh='p0.00',vvcrit='0.0',
                            model_ifmr = 'Cummings_2018_PARSEC',
                            high_perc = 84, low_perc = 16,
                            datatype='Gyr',
                            save_plots=False, display_plots=False)  #i'm going to edit them so I don't want to save/see them so many times
    WD.calc_wd_age()
    results = wd.calc_percentiles(10**WD.log10_tms_sample/1e9,
                                    10**WD.log10_tcool_sample/1e9,
                                    10**WD.log10_ttot_sample/1e9,
                                    WD.mi_sample, WD.mf_sample,
                                    WD.high_perc, WD.low_perc)   
    data.append(10**WD.log10_ttot_sample/1e9)
    print(i)

nondanew=[] 
for i in range(len(joint_nonda)):
    WD=wd.WhiteDwarf(joint_nonda.Teff.values,joint_nonda.e_Teff.values,joint_nonda.logg.values,joint_nonda.e_logg.values,
                            model_wd='DA',feh='p0.00',vvcrit='0.0',
                            model_ifmr = 'Cummings_2018_PARSEC',
                            high_perc = 84, low_perc = 16,
                            datatype='Gyr',
                            save_plots=False, display_plots=False)  #i'm going to edit them so I don't want to save/see them so many times
    WD.calc_wd_age()
    results = wd.calc_percentiles(10**WD.log10_tms_sample/1e9,
                                    10**WD.log10_tcool_sample/1e9,
                                    10**WD.log10_ttot_sample/1e9,
                                    WD.mi_sample, WD.mf_sample,
                                    WD.high_perc, WD.low_perc)   
    nondanew.append(10**WD.log10_ttot_sample/1e9)
    print(i)

data2= np.vstack([data, nondanew])

In [ ]:
#this plots the N=17 joint image! (as long as the da_srcs & nonda_srcs have been changed to the not nan, bw 300 & 750 Myr srcs)
plt.figure(figsize=(2.93, 2.7))
for i in range(len(data2)):
    plt.hist(data2[i], bins=20, alpha=0.3, color='#1f77b4')

plt.gca().axes.get_yaxis().set_visible(False)
plt.xlabel('Total Age (Gyr)')
plt.title('N=17')
plt.xlim(0.3,.7)
plt.show()

In [ ]:
#will have to play around with how to do this... need to save them as the age
title = r"${{{0:.2f}}}_{{-{1:.2f}}}^{{+{2:.2f}}}$"
f, (ax3) = plt.subplots(1, 1, figsize=(3, 3))

axs = [ax3]

labels = ['Total Age (Gyr)']

distributions = [10**WD.log10_ttot_sample/1e9]

for ax, label, dist, i, in zip(axs, labels, distributions, np.arange(0, 15, 3)):
    ax.hist(dist[~np.isnan(dist)], bins=20)
    ax.axvline(x=results[6], color='k')
    ax.axvline(x=results[6] - results[6+1], color='k', linestyle='--')
    ax.axvline(x=results[6] + results[6+2], color='k', linestyle='--')
    ax.set_xlabel(label)
    ax.yaxis.set_visible(False)
    ax.set_title(title.format(np.round(results[6], 2),
                              np.round(results[6+1], 2),
                              np.round(results[6+2], 2)))
    
ax3.set_xlim(0.3, 0.7)
plt.tight_layout()
plt.show()